In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Carregar o dataset
file_path = '/content/drive/MyDrive/bot_detection_data.csv'
df = pd.read_csv(file_path)

# Exibir as primeiras linhas e informações gerais do dataset
df.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User ID         50000 non-null  int64 
 1   Username        50000 non-null  object
 2   Tweet           50000 non-null  object
 3   Retweet Count   50000 non-null  int64 
 4   Mention Count   50000 non-null  int64 
 5   Follower Count  50000 non-null  int64 
 6   Verified        50000 non-null  bool  
 7   Bot Label       50000 non-null  int64 
 8   Location        50000 non-null  object
 9   Created At      50000 non-null  object
 10  Hashtags        41659 non-null  object
dtypes: bool(1), int64(5), object(5)
memory usage: 3.9+ MB


In [ ]:
# Verificar a distribuição das classes (bot vs. humano)
print(df['Bot Label'].value_counts())
# Verificar se há colunas não numéricas
print(df.select_dtypes(include=['object']).head())

from sklearn.preprocessing import LabelEncoder


# Tratar colunas categóricas
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Separar features e label (alvo)
X = df.drop(columns=['Bot Label'])
y = df['Bot Label']

# Padronização das features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar SMOTE para balancear as classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Ajuste de hiperparâmetros usando GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado pelo GridSearch
best_rf_model = grid_search.best_estimator_

# Fazer previsões nos dados de teste
y_pred = best_rf_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')
print(classification_report(y_test, y_pred))

# Exibir os melhores hiperparâmetros
print(f'Melhores hiperparâmetros: {grid_search.best_params_}')

Bot Label
1    25018
0    24982
Name: count, dtype: int64
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]
Fitting 3 folds for each of 216 candidates, totalling 648 fits


# 1. Verificar a distribuição das classes


```
print(df['Bot Label'].value_counts())
```


Objetivo: Verificar quantos exemplos existem para cada classe no rótulo (bot vs. humano) para entender se há um desbalanceamento nas classes.

# 2. Verificar se há colunas não numéricas

```
print(df.select_dtypes(include=['object']).head())
```  


Objetivo: Identificar colunas que contenham dados não numéricos, como strings, para tratar esses dados adequadamente antes de modelar.

# 3. Tratar colunas categóricas


```
from sklearn.preprocessing import LabelEncoder
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le
    
```


Objetivo: Converter as colunas categóricas (texto) em valores numéricos utilizando LabelEncoder. Isso é necessário porque os algoritmos de machine learning geralmente aceitam apenas valores numéricos.

# 4. Separar as features e o rótulo (alvo)

```
X = df.drop(columns=['Bot Label'])
y = df['Bot Label']
```


Objetivo: Separar os dados de entrada (X), que são as variáveis preditoras, e o rótulo (variável alvo, y), que indica se a conta é um bot ou humano.

# 5. Padronizar as features


```
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
```


Objetivo: Padronizar os dados de entrada, de forma que todas as variáveis tenham a mesma escala, o que pode melhorar o desempenho de alguns algoritmos de machine learning.

# 6. Aplicar SMOTE para balanceamento das classes

```
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)
```


Objetivo: Usar o SMOTE (Synthetic Minority Over-sampling Technique) para balancear as classes criando exemplos sintéticos para a classe minoritária (se houver desbalanceamento).

# 7. Dividir os dados em treino e teste

```
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
```


Objetivo: Dividir os dados em conjuntos de treino e teste, reservando 20% dos dados para testar o desempenho do modelo.

# 8. Ajuste de hiperparâmetros com GridSearchCV

```
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
```

Objetivo: Usar GridSearchCV para encontrar os melhores hiperparâmetros para o RandomForestClassifier por meio de uma busca exaustiva em um grid de combinações de parâmetros.

# 9. Obter o melhor modelo encontrado

```
best_rf_model = grid_search.best_estimator_
```


Objetivo: Capturar o melhor modelo encontrado pela busca de hiperparâmetros.

# 10. Fazer previsões nos dados de teste

```
y_pred = best_rf_model.predict(X_test)
```


Objetivo: Usar o modelo treinado para fazer previsões no conjunto de teste.

# 11. Avaliar o desempenho do modelo

```
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')
print(classification_report(y_test, y_pred))
```


Objetivo: Avaliar o desempenho do modelo utilizando métricas como acurácia e um relatório de classificação que inclui precisão, recall e F1-score.

# 12. Exibir os melhores hiperparâmetros

```
print(f'Melhores hiperparâmetros: {grid_search.best_params_}')
```


Objetivo: Exibir os melhores hiperparâmetros encontrados durante a busca.